In [ ]:
import numpy as np

import h5py

import h5flow
from h5flow.data import dereference

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

import sys
import os

# import  moviepy.editor as mpy
import io 
from PIL import Image

In [ ]:
def divide_closest(n, d):
    return (n + d // 2) // d

In [ ]:
def draw_cathode_planes(x_boundaries, y_boundaries, z_boundaries, **kwargs):
    
    traces = []
    for i_z in range(int(len(z_boundaries)/2)):
        for i_x in range(int(len(x_boundaries)/2)):
            z, y = np.meshgrid(np.linspace(z_boundaries[i_z * 2], z_boundaries[i_z * 2 + 1], 2), np.linspace(y_boundaries.min(), y_boundaries.max(),2))
            x = (x_boundaries[i_x * 2] + x_boundaries[i_x * 2 + 1]) * 0.5 * np.ones(z.shape)

            traces.append(go.Surface(x=x, y=y, z=z, **kwargs))

    return traces

def draw_anode_planes(x_boundaries, y_boundaries, z_boundaries, **kwargs):
    
    traces = []
    for i_z in range(int(len(z_boundaries)/2)):
        for i_x in range(int(len(x_boundaries))):           
            z, y = np.meshgrid(np.linspace(z_boundaries[i_z * 2], z_boundaries[i_z * 2 + 1], 2), np.linspace(y_boundaries.min(), y_boundaries.max(),2))
            x = x_boundaries[i_x] * np.ones(z.shape)

            traces.append(go.Surface(x=x, y=y, z=z, **kwargs))
    
    return traces

In [ ]:
def hit_xmax_edge(hit_x, hit_y, hit_z):
    if (np.max(hit_x) > np.max(x_boundaries) - edge_threshold) and (np.max(hit_x) < np.max(x_boundaries) + edge_threshold): 
        edge_idx = np.argmax(hit_x)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max
    
def hit_xmin_edge(hit_x, hit_y, hit_z):
    if (np.min(hit_x) > np.min(x_boundaries) - edge_threshold) and (np.min(hit_x) < np.min(x_boundaries) + edge_threshold): 
        edge_idx = np.argmin(hit_x)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False    
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max
    
def hit_ymax_edge(hit_x, hit_y, hit_z):
    if (np.max(hit_y) > np.max(y_boundaries) - edge_threshold) and (np.max(hit_y) < np.max(y_boundaries) + edge_threshold): 
        edge_idx = np.argmax(hit_y)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max
    
def hit_ymin_edge(hit_x, hit_y, hit_z):
    if (np.min(hit_y) > np.min(y_boundaries) - edge_threshold) and (np.min(hit_y) < np.min(y_boundaries) + edge_threshold): 
        edge_idx = np.argmin(hit_y)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max
    
def hit_zmax_edge(hit_x, hit_y, hit_z):
    if (np.max(hit_z) > np.max(z_boundaries) - edge_threshold) and (np.max(hit_z) < np.max(z_boundaries) + edge_threshold): 
        edge_idx = np.argmax(hit_z)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max
    
def hit_zmin_edge(hit_x, hit_y, hit_z):
    if (np.min(hit_z) > np.min(z_boundaries) - edge_threshold) and (np.min(hit_z) < np.min(z_boundaries) + edge_threshold): 
        edge_idx = np.argmin(hit_z)
        return hit_x[edge_idx], hit_y[edge_idx], hit_z[edge_idx]
    else:
        return False
        # return sys.float_info.max, sys.float_info.max, sys.float_info.max

def hit_cathode_xplus(hit_x, hit_y, hit_z):
    if len(hit_x[hit_x>0])> 0:
        if np.min(hit_x[hit_x>0]) < edge_threshold:
            edge_idx = np.argmin(hit_x[hit_x>0])
            return hit_x[hit_x>0][edge_idx], hit_y[hit_x>0][edge_idx], hit_z[hit_x>0][edge_idx]
    else:
        return False

def hit_cathode_xminus(hit_x, hit_y, hit_z):
    if len(hit_x[hit_x<0])> 0:
        if np.max(hit_x[hit_x<0]) > -edge_threshold:
            edge_idx = np.argmax(hit_x[hit_x<0])
            return hit_x[hit_x<0][edge_idx], hit_y[hit_x<0][edge_idx], hit_z[hit_x<0][edge_idx]
    else:
        return False
       
def out_time(hit_x, hit_y, hit_z):
    if (np.max(hit_x) >= np.max(x_boundaries) + edge_threshold) or (np.min(hit_x) <= np.min(x_boundaries) - edge_threshold):
        return True
    
    if hit_xmax_edge(hit_x, hit_y, hit_z):
        hit_xmax_x, hit_xmax_y, hit_xmax_z = hit_xmax_edge(hit_x, hit_y, hit_z)
        if hit_xmax_y < np.max(y_boundaries)/2:
            return True        
    return False

def hit_edge(hit_x, hit_y, hit_z):
    hit_edge = []
    if out_time(hit_x, hit_y, hit_z):
        return False
    if hit_xmax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_xmax_edge(hit_x, hit_y, hit_z)))
    if hit_xmin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_xmin_edge(hit_x, hit_y, hit_z)))
    if hit_ymax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_ymax_edge(hit_x, hit_y, hit_z)))
    if hit_ymin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_ymin_edge(hit_x, hit_y, hit_z)))
    if hit_zmax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_zmax_edge(hit_x, hit_y, hit_z)))
    if hit_zmin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_zmin_edge(hit_x, hit_y, hit_z)))
        
    if len(hit_edge) > 0:
        return hit_edge
    else:
        return False
    
def hit_tpc_edge(hit_x, hit_y, hit_z):
    hit_edge = []
    if out_time(hit_x, hit_y, hit_z):
        return False
    if hit_xmax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_xmax_edge(hit_x, hit_y, hit_z)))
    if hit_xmin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_xmin_edge(hit_x, hit_y, hit_z)))
    if hit_ymax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_ymax_edge(hit_x, hit_y, hit_z)))
    if hit_ymin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_ymin_edge(hit_x, hit_y, hit_z)))
    if hit_zmax_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_zmax_edge(hit_x, hit_y, hit_z)))
    if hit_zmin_edge(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_zmin_edge(hit_x, hit_y, hit_z)))
    if hit_cathode_xplus(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_cathode_xplus(hit_x, hit_y, hit_z)))
    if hit_cathode_xminus(hit_x, hit_y, hit_z):
        hit_edge.append(np.array(hit_cathode_xminus(hit_x, hit_y, hit_z)))
        
    if len(hit_edge) > 0:
        return hit_edge
    else:
        return False
    
def hit_ct_end(hit_x, hit_y, hit_z):
    ct_end = []
    end1 = np.argmax(hit_x)
    end2 = np.argmin(hit_x)
    ct_end.append([hit_x[end1], hit_y[end1], hit_z[end1]])    
    ct_end.append([hit_x[end2], hit_y[end2], hit_z[end2]])
    return ct_end

In [ ]:
x_boundaries = np.array([-30.431, 30.431])
y_boundaries = np.array([-61.8543, 61.8543])
z_boundaries = np.array([-30.8163, 30.8163])

In [ ]:
edge_threshold = 4
ct_hits_threshold = 10
crazy_n_hits_threshold = 5000
seg_n_hits = 30
evt_hits_threshold = 200
edge_ROI = 15

In [ ]:
f_name = '/global/cfs/cdirs/dune/www/data/ModuleX/flow/Run_0-5_Efield_1/flow_selftrigger_packet_2023_10_04_16_07_CEST.h5'
f_manager = h5flow.data.H5FlowDataManager(f_name, 'r')

In [ ]:
# edge_hits = []
# edge_tpc_hits = []

i_evts = []
edge_xmax_hits = []
edge_xmin_hits = []
edge_ymax_hits = []
edge_ymin_hits = []
edge_zmax_hits = []
edge_zmin_hits = []
edge_cathode_xplus_hits = []
edge_cathode_xminus_hits = []

evt_crossing_track = []

ct_fit_length = []
ct_PCA_mean = []
ct_PCA_vec = []
ct_PCA_variance = []
ct_PCA_singular_values = []
ct_pts_density = []

seg_fit_length = []
seg_PCA_mean = []
seg_PCA_vec = []
seg_PCA_variance = []
seg_PCA_singular_values = []
seg_pts_density = []

seg_cos_angle = []
seg_angle = []
seg_joint = []

ct_ends = []

f_n_evt = f_manager["charge/events/data"]

for i_evt in f_n_evt['id']:
    if i_evt % 5000 == 0:
        print(i_evt)
    if f_n_evt['nhit'][i_evt] < evt_hits_threshold:
        continue
    if f_n_evt['nhit'][i_evt] > crazy_n_hits_threshold:
        continue
        
    PromptHits_ev = f_manager["charge/events", "charge/calib_prompt_hits", i_evt]
    this_hit_x, this_hit_y, this_hit_z = PromptHits_ev.data['x'].flatten(), PromptHits_ev.data['y'].flatten(), PromptHits_ev.data['z'].flatten()
    nan_mask = np.isfinite(this_hit_x) & np.isfinite(this_hit_y) & np.isfinite(this_hit_z)
    hit_x = this_hit_x[nan_mask]
    hit_y = this_hit_y[nan_mask]
    hit_z = this_hit_z[nan_mask]
    pts = np.array([hit_x, hit_y, hit_z]).T 
    
    ymax_idx = np.argmax(hit_y)
    ymin_idx = np.argmin(hit_y)
    
#     if (max(hit_y) - min(hit_y)) > 80 and hit_z[ymax_idx] < (min(z_boundaries) + edge_ROI) and hit_z[ymin_idx] < (min(z_boundaries) + edge_ROI):
#         # print("max(hit_y): ", max(hit_y))
#         # print("min(hit_y): ", min(hit_y))
#         # print("hit_z[ymax_idx]: ", hit_x[ymax_idx], hit_y[ymax_idx], hit_z[ymax_idx])
#         # print("hit_z[ymin_idx]: ", hit_x[ymin_idx], hit_y[ymin_idx], hit_z[ymin_idx])
#         # print("(min(z_boundaries) + edge_ROI): ", (min(z_boundaries) + edge_ROI))
#         print("A---ievt: ", i_evt)    
#     elif max(hit_y) > (max(y_boundaries) - edge_ROI) and min(hit_y) > (max(y_boundaries) - edge_ROI):
#         print("B---ievt: ", i_evt)
#     elif max(hit_y) < (min(y_boundaries) + edge_ROI) and min(hit_y) > (min(y_boundaries) + edge_ROI):
#         print("C---ievt: ", i_evt)
#     else:
#         continue
    
    if (max(hit_y) - min(hit_y)) > 80 and hit_z[ymax_idx] > (min(z_boundaries) + edge_ROI) and hit_z[ymin_idx] > (min(z_boundaries) + edge_ROI):
        print("A---ievt: ", i_evt)    
        
    i_evts.append(i_evt)
    
    db = DBSCAN(eps=5, min_samples=3).fit(pts)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    if hit_xmax_edge(hit_x, hit_y, hit_z) and hit_xmin_edge(hit_x, hit_y, hit_z):
        evt_crossing_track.append(i_evt)

    for i_ct in range(n_clusters_):
        ct_mask = labels == i_ct
        hit_x_ct = hit_x[ct_mask]
        hit_y_ct = hit_y[ct_mask]
        hit_z_ct = hit_z[ct_mask]
        
        n_hit_ct = np.count_nonzero(ct_mask)
        
        if out_time(hit_x_ct, hit_y_ct, hit_z_ct):
            continue
        if n_hit_ct < ct_hits_threshold:
            continue
    
        #####################
        # Fill edge most hits
        #####################
        if hit_xmax_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_xmax_hits.append(hit_xmax_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_xmin_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_xmin_hits.append(hit_xmin_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_ymax_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_ymax_hits.append(hit_ymax_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_ymin_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_ymin_hits.append(hit_ymin_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_zmax_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_zmax_hits.append(hit_zmax_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_zmin_edge(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_zmin_hits.append(hit_zmin_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_cathode_xplus(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_cathode_xplus_hits.append(hit_cathode_xplus(hit_x_ct, hit_y_ct, hit_z_ct))
        if hit_cathode_xminus(hit_x_ct, hit_y_ct, hit_z_ct):
            edge_cathode_xminus_hits.append(hit_cathode_xminus(hit_x_ct, hit_y_ct, hit_z_ct))
        
#         if hit_edge(hit_x_ct, hit_y_ct, hit_z_ct):
#             edge_hits.append(hit_edge(hit_x_ct, hit_y_ct, hit_z_ct))
        
#         if hit_tpc_edge(hit_x_ct, hit_y_ct, hit_z_ct):
#             edge_tpc_hits.append(hit_tpc_edge(hit_x_ct, hit_y_ct, hit_z_ct))
            
        #####################
        # Fill end of cluster hits
        #####################
        this_ct_ends = hit_ct_end(hit_x_ct, hit_y_ct, hit_z_ct)
        for i_end in range(len(this_ct_ends)):
            ct_ends.append(this_ct_ends[i_end])           
            # # print("this_ct_ends[i_end]: ", this_ct_ends[i_end])
            # # print("edge_tpc_hits: ", edge_tpc_hits)
            # if this_ct_ends[i_end] not in np.concatenate(edge_tpc_hits):
            # # if this_ct_ends[i_end] not in np.concatenate(edge_tpc_hits):
            #     ct_ends.append(this_ct_ends[i_end])
                
        #####################
        # PCA
        #####################
        ct_pts = np.array([hit_x_ct, hit_y_ct, hit_z_ct]).T
        pca = PCA(n_components=2)
        pca.fit(ct_pts)
        
        centre = pca.mean_
        zmax_idx = np.argmax(hit_z_ct)
        zmin_idx = np.argmin(hit_z_ct)
        length = np.linalg.norm([(hit_x_ct[zmax_idx]-hit_x_ct[zmin_idx]),(hit_y_ct[zmax_idx]-hit_y_ct[zmin_idx]),(hit_z_ct[zmax_idx]-hit_z_ct[zmin_idx])]) # estimated length, line assumption
        end1 = centre + 0.5 * length * pca.components_[0]
        end2 = centre - 0.5 * length * pca.components_[0]
       
        centre = pca.mean_
        zmax_idx = np.argmax(hit_z_ct)
        zmin_idx = np.argmin(hit_z_ct)
        length_z = np.linalg.norm([(hit_x_ct[zmax_idx]-hit_x_ct[zmin_idx]),(hit_y_ct[zmax_idx]-hit_y_ct[zmin_idx]),(hit_z_ct[zmax_idx]-hit_z_ct[zmin_idx])]) # estimated length, line assumption
        ymax_idx = np.argmax(hit_y_ct)
        ymin_idx = np.argmin(hit_y_ct)
        length_y = np.linalg.norm([(hit_x_ct[ymax_idx]-hit_x_ct[ymin_idx]),(hit_y_ct[ymax_idx]-hit_y_ct[ymin_idx]),(hit_z_ct[ymax_idx]-hit_z_ct[ymin_idx])]) # estimated length, line assumption
        xmax_idx = np.argmax(hit_x_ct)
        xmin_idx = np.argmin(hit_x_ct)
        length_x = np.linalg.norm([(hit_x_ct[xmax_idx]-hit_x_ct[xmin_idx]),(hit_y_ct[xmax_idx]-hit_y_ct[xmin_idx]),(hit_z_ct[xmax_idx]-hit_z_ct[xmin_idx])]) # estimated length, line assumption
        length = np.max([length_x, length_y, length_z])
        end1 = centre + 0.5 * length * pca.components_[0]
        end2 = centre - 0.5 * length * pca.components_[0]
        
        ct_fit_length.append(length)
        ct_PCA_mean.append(centre)
        ct_PCA_vec.append(pca.components_[0])
        ct_PCA_variance.append(pca.explained_variance_)
        ct_PCA_singular_values.append(pca.singular_values_)
        ct_pts_density.append(n_hit_ct/length)
        
        ## fit segments
        sort_x_idx = np.argsort(hit_x_ct)
        ct_pts_sort_x = ct_pts[sort_x_idx]
        for i_seg in range(divide_closest(n_hit_ct, seg_n_hits)):
            start_idx = seg_n_hits * i_seg
            if i_seg == (divide_closest(n_hit_ct, seg_n_hits) - 1):
                end_idx = max(seg_n_hits * (i_seg+1), len(ct_pts_sort_x))
            else:
                end_idx = seg_n_hits * (i_seg+1)

            seg_pts = ct_pts_sort_x[start_idx: end_idx]
            pca_seg = PCA(n_components=2)
            pca_seg.fit(seg_pts)

            seg_centre = pca_seg.mean_
            seg_zmax_idx = np.argmax(seg_pts[:,2])
            seg_zmin_idx = np.argmin(seg_pts[:,2])
            seg_length_z = np.linalg.norm([(seg_pts[:,0][seg_zmax_idx]-seg_pts[:,0][seg_zmin_idx]),(seg_pts[:,1][seg_zmax_idx]-seg_pts[:,1][seg_zmin_idx]),(seg_pts[:,2][seg_zmax_idx]-seg_pts[:,2][seg_zmin_idx])]) # estimated length, line assumption 
            seg_ymax_idx = np.argmax(seg_pts[:,1])
            seg_ymin_idx = np.argmin(seg_pts[:,1])
            seg_length_y = np.linalg.norm([(seg_pts[:,0][seg_ymax_idx]-seg_pts[:,0][seg_ymin_idx]),(seg_pts[:,1][seg_ymax_idx]-seg_pts[:,1][seg_ymin_idx]),(seg_pts[:,2][seg_ymax_idx]-seg_pts[:,2][seg_ymin_idx])]) # estimated length, line assumption
            seg_xmax_idx = np.argmax(seg_pts[:,0])
            seg_xmin_idx = np.argmin(seg_pts[:,0])
            seg_length_x = np.linalg.norm([(seg_pts[:,0][seg_xmax_idx]-seg_pts[:,0][seg_xmin_idx]),(seg_pts[:,1][seg_xmax_idx]-seg_pts[:,1][seg_xmin_idx]),(seg_pts[:,2][seg_xmax_idx]-seg_pts[:,2][seg_xmin_idx])]) # estimated length, line assumption
            seg_length = np.max([length_x, length_y, length_z])
            seg_end1 = seg_centre + 0.5 * seg_length_z * pca_seg.components_[0]
            seg_end2 = seg_centre - 0.5 * seg_length_z * pca_seg.components_[0]
            
            seg_fit_length.append(seg_length)
            seg_PCA_mean.append(seg_centre)
            seg_PCA_vec.append(pca_seg.components_[0])
            seg_PCA_variance.append(pca_seg.explained_variance_)
            seg_PCA_singular_values.append(pca_seg.singular_values_)
            seg_pts_density.append(len(seg_pts)/seg_length)
            
            if i_seg > 0:
                cos_angle = abs(np.sum(seg_PCA_vec[i_seg] * seg_PCA_vec[i_seg-1])/(np.linalg.norm(seg_PCA_vec[i_seg])*np.linalg.norm(seg_PCA_vec[i_seg-1])))
                angle = np.arccos(cos_angle)/np.pi * 180
                # angle = np.min([np.arccos(cos_angle)/np.pi * 180, 180 - np.arccos(cos_angle)/np.pi * 180])
                seg_cos_angle.append(cos_angle)
                seg_angle.append(angle)
                seg_joint.append((seg_PCA_mean[i_seg]+seg_PCA_mean[i_seg-1])/2)

i_evts = np.array(i_evts)
edge_xmax_hits = np.array(edge_xmax_hits) 
edge_xmin_hits = np.array(edge_xmin_hits) 
edge_ymax_hits = np.array(edge_ymax_hits) 
edge_ymin_hits = np.array(edge_ymin_hits)
edge_zmax_hits = np.array(edge_zmax_hits)
edge_zmin_hits = np.array(edge_zmin_hits) 
edge_cathode_xplus_hits = np.array(edge_cathode_xplus_hits) 
edge_cathode_xminus_hits = np.array(edge_cathode_xminus_hits) 

# edge_hits = np.concatenate(edge_hits)
# edge_tpc_hits = np.concatenate(edge_tpc_hits)

evt_crossing_track = np.array(evt_crossing_track)

ct_fit_length = np.array(ct_fit_length)
ct_PCA_mean = np.array(ct_PCA_mean)
ct_PCA_vec = np.array(ct_PCA_vec)
ct_PCA_variance = np.array(ct_PCA_variance)
ct_PCA_singular_values = np.array(ct_PCA_singular_values)
ct_pts_density = np.array(ct_pts_density)

seg_fit_length = np.array(seg_fit_length)
seg_PCA_mean = np.array(seg_PCA_mean)
seg_PCA_vec = np.array(seg_PCA_vec)
seg_PCA_variance = np.array(seg_PCA_variance)
seg_PCA_singular_values = np.array(seg_PCA_singular_values)
seg_pts_density = np.array(seg_pts_density)

seg_cos_angle = np.array(seg_cos_angle)
seg_angle = np.array(seg_angle)
seg_joint = np.array(seg_joint)
                
ct_ends = np.array(ct_ends)

# np.savez(outfile, 
#     edge_xmax_hits = edge_xmax_hits,
#     edge_xmin_hits = edge_xmin_hits,
#     edge_ymax_hits = edge_ymax_hits,
#     edge_ymin_hits = edge_ymin_hits,
#     edge_zmax_hits = edge_zmax_hits,
#     edge_zmin_hits = edge_zmin_hits,
#     edge_cathode_xplus_hits = edge_cathode_xplus_hits,
#     edge_cathode_xminus_hits = edge_cathode_xminus_hits,
#     evt_crossing_track = evt_crossing_track,
#     ct_fit_length = ct_fit_length,
#     ct_PCA_mean = ct_PCA_mean,
#     ct_PCA_vec = ct_PCA_vec,
#     ct_PCA_variance = ct_PCA_variance,
#     ct_PCA_singular_values = ct_PCA_singular_values,
#     ct_pts_density = ct_pts_density,
#     seg_fit_length = seg_fit_length,
#     seg_PCA_mean = seg_PCA_mean,
#     seg_PCA_vec = seg_PCA_vec,
#     seg_PCA_variance = seg_PCA_variance,
#     seg_PCA_singular_values = seg_PCA_singular_values,
#     seg_pts_density = seg_pts_density,
#     seg_cos_angle = seg_cos_angle,
#     seg_angle = seg_angle,
#     seg_joint = seg_joint,
#     ct_ends = ct_ends)

In [ ]:
f = np.load("/global/cfs/cdirs/dune/www/data/ModuleX/analysis_data/analysis_0-5_Efield_1/hit_seg_edge_2023_10_04_16_07_CEST.npz")
edge_xmax_hits = f["edge_xmax_hits"]
edge_xmin_hits = f["edge_xmin_hits"]
edge_ymax_hits = f["edge_ymax_hits"]
edge_ymin_hits = f["edge_ymin_hits"]
edge_zmax_hits = f["edge_zmax_hits"]
edge_zmin_hits = f["edge_zmin_hits"]
edge_cathode_xplus_hits = f["edge_cathode_xplus_hits"]
edge_cathode_xminus_hits = f["edge_cathode_xminus_hits"]
ct_ends = f['ct_ends']

In [ ]:
#2342 #4515 #4673 #174
# i_evt = 21806 # 2023_10_04_07_55_CEST crossing: 370, 6300, 16472, 21806, 25640, 14406
i_evt = 5111 # 2023_10_04_16_07_CEST crossing: 1907, 3173, 3535, *3785*, 4588, 6022, 6863, *7096*, *7316*,8927, 11261, *30524*, *26071*, *30163*, *27866*
PromptHits_ev = f_manager["charge/events", "charge/calib_prompt_hits", i_evt]

this_hit_x, this_hit_y, this_hit_z = PromptHits_ev.data['x'].flatten(), PromptHits_ev.data['y'].flatten(), PromptHits_ev.data['z'].flatten()
nan_mask = np.isfinite(this_hit_x) & np.isfinite(this_hit_y) & np.isfinite(this_hit_z)
hit_x = this_hit_x[nan_mask]
hit_y = this_hit_y[nan_mask]
hit_z = this_hit_z[nan_mask]

pts = np.array([hit_x, hit_y, hit_z]).T 


db = DBSCAN(eps=5, min_samples=3).fit(pts)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
            
fig = go.Figure()

##########################
# Draw the cathodes
##########################
fig.add_traces(draw_cathode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.3,
    colorscale='Greys',
))

##########################
# Draw the anodes
##########################
fig.add_traces(draw_anode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

##########################
# Draw the prompt hits
##########################
PHits_traces = go.Scatter3d(
        x=hit_x, y=hit_y, z=hit_z,
        # x=PromptHits_ev.data['x'].flatten(), y=PromptHits_ev.data['y'].flatten(), z=PromptHits_ev.data['z'].flatten(),
        marker_color=PromptHits_ev.data['E'].flatten()*1000,
        name='prompt hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=PromptHits_ev.data['E'].flatten()*1000,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>E:%{customdata:.3f}',
        )

##########################
# DBScan and PCA
##########################
PCA_mean = []
PCA_line = []
ct_hits = []
seg_hits = []
seg_PCA_line = []
seg_PCA_mean = []

seg_vec = []
for i_ct in range(n_clusters_):
    ct_mask = labels == i_ct
    hit_x_ct = hit_x[ct_mask]
    hit_y_ct = hit_y[ct_mask]
    hit_z_ct = hit_z[ct_mask]

    n_hit_ct = np.count_nonzero(ct_mask)
    if out_time(hit_x_ct, hit_y_ct, hit_z_ct):
        continue
    if np.count_nonzero(ct_mask) < ct_hits_threshold:
        continue

    ct_pts = np.array([hit_x_ct, hit_y_ct, hit_z_ct]).T
    pca = PCA(n_components=2)
    pca.fit(ct_pts)
    print(pca.singular_values_)
    print(pca.components_)
    print(pca.explained_variance_)
    
    centre = pca.mean_
    zmax_idx = np.argmax(hit_z_ct)
    zmin_idx = np.argmin(hit_z_ct)
    length_z = np.linalg.norm([(hit_x_ct[zmax_idx]-hit_x_ct[zmin_idx]),(hit_y_ct[zmax_idx]-hit_y_ct[zmin_idx]),(hit_z_ct[zmax_idx]-hit_z_ct[zmin_idx])]) # estimated length, line assumption
    ymax_idx = np.argmax(hit_y_ct)
    ymin_idx = np.argmin(hit_y_ct)
    length_y = np.linalg.norm([(hit_x_ct[ymax_idx]-hit_x_ct[ymin_idx]),(hit_y_ct[ymax_idx]-hit_y_ct[ymin_idx]),(hit_z_ct[ymax_idx]-hit_z_ct[ymin_idx])]) # estimated length, line assumption
    xmax_idx = np.argmax(hit_x_ct)
    xmin_idx = np.argmin(hit_x_ct)
    length_x = np.linalg.norm([(hit_x_ct[xmax_idx]-hit_x_ct[xmin_idx]),(hit_y_ct[xmax_idx]-hit_y_ct[xmin_idx]),(hit_z_ct[xmax_idx]-hit_z_ct[xmin_idx])]) # estimated length, line assumption
    length = np.max([length_x, length_y, length_z])
    # axis_idx = np.argmax([length_x, length_y, length_z])
    # length = pca.singular_values_[0] # line assumption
    end1 = centre + 0.5 * length * pca.components_[0]
    end2 = centre - 0.5 * length * pca.components_[0]
    
    PCA_mean.append(go.Scatter3d(
        x=[centre[0]], y=[centre[1]], z=[centre[2]],
        marker_color='blue',
        name=f'PCA mean (cluster {i_ct})',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>'
    ))
    
    PCA_line.append(go.Scatter3d(
        x=[end1[0], end2[0]], y=[end1[1], end2[1]], z=[end1[2], end2[2]],
        line_color='blue',
        name=f'PCA line (cluster {i_ct})',
        mode='lines',
        visible='legendonly',
        # marker_size=3,
        # marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>'
    ))
    
    ct_hits.append(go.Scatter3d(
        x=hit_x_ct, y=hit_y_ct, z=hit_z_ct,
        # x=PromptHits_ev.data['x'].flatten(), y=PromptHits_ev.data['y'].flatten(), z=PromptHits_ev.data['z'].flatten(),
        marker_color=i_ct,
        name=f'hits (cluster {i_ct})',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        # customdata=PromptHits_ev.data['E'].flatten()*1000,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>',
    ))
    
    ## fit every 30 hits
    sort_x_idx = np.argsort(hit_x_ct)
    ct_pts_sort_x = ct_pts[sort_x_idx]
    # print(len(ct_pts))
    # print(len(ct_pts_sort_x))
    for i_seg in range(divide_closest(n_hit_ct, seg_n_hits)):
        start_idx = seg_n_hits * i_seg
        end_idx = seg_n_hits * (i_seg+1)
        
        seg_pts = ct_pts_sort_x[start_idx: end_idx]
        pca_seg = PCA(n_components=2)
        pca_seg.fit(seg_pts)
        seg_vec.append(pca_seg.components_[0])
        # print(pca_seg.components_[0])
        
        seg_centre = pca_seg.mean_
        seg_zmax_idx = np.argmax(seg_pts[:,2])
        seg_zmin_idx = np.argmin(seg_pts[:,2])
        seg_length_z = np.linalg.norm([(seg_pts[:,0][seg_zmax_idx]-seg_pts[:,0][seg_zmin_idx]),(seg_pts[:,1][seg_zmax_idx]-seg_pts[:,1][seg_zmin_idx]),(seg_pts[:,2][seg_zmax_idx]-seg_pts[:,2][seg_zmin_idx])]) # estimated length, line assumption 
        seg_ymax_idx = np.argmax(seg_pts[:,1])
        seg_ymin_idx = np.argmin(seg_pts[:,1])
        seg_length_y = np.linalg.norm([(seg_pts[:,0][seg_ymax_idx]-seg_pts[:,0][seg_ymin_idx]),(seg_pts[:,1][seg_ymax_idx]-seg_pts[:,1][seg_ymin_idx]),(seg_pts[:,2][seg_ymax_idx]-seg_pts[:,2][seg_ymin_idx])]) # estimated length, line assumption
        seg_xmax_idx = np.argmax(seg_pts[:,0])
        seg_xmin_idx = np.argmin(seg_pts[:,0])
        seg_length_x = np.linalg.norm([(seg_pts[:,0][seg_xmax_idx]-seg_pts[:,0][seg_xmin_idx]),(seg_pts[:,1][seg_xmax_idx]-seg_pts[:,1][seg_xmin_idx]),(seg_pts[:,2][seg_xmax_idx]-seg_pts[:,2][seg_xmin_idx])]) # estimated length, line assumption
        seg_length = np.max([length_x, length_y, length_z])
        seg_end1 = seg_centre + 0.5 * seg_length_z * pca_seg.components_[0]
        seg_end2 = seg_centre - 0.5 * seg_length_z * pca_seg.components_[0]
        # print("density: ", n_hit_ct/seg_length_z)
        seg_hits.append(go.Scatter3d(
            x=seg_pts[:,0], y=seg_pts[:,1], z=seg_pts[:,2],
            marker_color=i_seg,
            name=f'hits (seg {i_seg})',
            mode='markers',
            visible='legendonly',
            marker_size=3,
            marker_symbol='square',
            showlegend=True,
            opacity=0.7,
            # customdata=PromptHits_ev.data['E'].flatten()*1000,
            hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>',
        ))
        
        seg_PCA_line.append(go.Scatter3d(
            x=[seg_end1[0], seg_end2[0]], y=[seg_end1[1], seg_end2[1]], z=[seg_end1[2], seg_end2[2]],
            line_color='red',
            name=f'seg PCA line (seg {i_seg})',
            mode='lines',
            visible='legendonly',
            # marker_size=3,
            # marker_symbol='square',
            showlegend=True,
            opacity=0.7,
            hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>'
        ))
        
        seg_PCA_mean.append(go.Scatter3d(
            x=[centre[0]], y=[centre[1]], z=[centre[2]],
            marker_color='blue',
            name=f'seg PCA mean (seg {i_seg})',
            mode='markers',
            visible='legendonly',
            marker_size=3,
            marker_symbol='square',
            showlegend=True,
            opacity=0.7,
            hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>'
        ))
        


PHits_xmax_traces = go.Scatter3d(
        x=edge_xmax_hits[:, 0], y=edge_xmax_hits[:, 1], z=edge_xmax_hits[:, 2],
        # marker_color='blue',
        name='edge xmax hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=np.max(x_boundaries)-edge_xmax_hits[:, 0],
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_xmax_traces)

PHits_xmin_traces = go.Scatter3d(
        x=edge_xmin_hits[:, 0], y=edge_xmin_hits[:, 1], z=edge_xmin_hits[:, 2],
        # marker_color='blue',
        name='edge xmin hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=edge_xmin_hits[:, 0]-np.min(x_boundaries),
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_xmin_traces)

PHits_ymax_traces = go.Scatter3d(
        x=edge_ymax_hits[:, 0], y=edge_ymax_hits[:, 1], z=edge_ymax_hits[:, 2],
        # marker_color='blue',
        name='edge ymax hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=np.max(y_boundaries)-edge_ymax_hits[:, 1],
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_ymax_traces)

PHits_ymin_traces = go.Scatter3d(
        x=edge_ymin_hits[:, 0], y=edge_ymin_hits[:, 1], z=edge_ymin_hits[:, 2],
        # marker_color='blue',
        name='edge ymin hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=edge_ymin_hits[:, 1]-np.min(y_boundaries),
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_ymin_traces)

# PHits_zmax_traces = go.Scatter3d(
#         x=edge_zmax_hits[:, 0], y=edge_zmax_hits[:, 1], z=edge_zmax_hits[:, 2],
#         # marker_color='blue',
#         name='edge zmax hits',
#         mode='markers',
#         visible='legendonly',
#         marker_size=3,
#         marker_symbol='square',
#         showlegend=True,
#         opacity=0.7,
#         customdata=np.max(z_boundaries)-edge_zmax_hits[:, 0],
#         hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
#         )
# fig.add_traces(PHits_zmax_traces)

PHits_zmin_traces = go.Scatter3d(
        x=edge_zmin_hits[:, 0], y=edge_zmin_hits[:, 1], z=edge_zmin_hits[:, 2],
        # marker_color='blue',
        name='edge zmin hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=edge_zmin_hits[:, 2]-np.min(z_boundaries),
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_zmin_traces)

PHits_cathode_xplus_traces = go.Scatter3d(
        x=edge_cathode_xplus_hits[:, 0], y=edge_cathode_xplus_hits[:, 1], z=edge_cathode_xplus_hits[:, 2],
        # marker_color='blue',
        name='edge cathode xplus hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=edge_cathode_xplus_hits[:, 0],
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_cathode_xplus_traces)

PHits_cathode_xminus_traces = go.Scatter3d(
        x=edge_cathode_xminus_hits[:, 0], y=edge_cathode_xminus_hits[:, 1], z=edge_cathode_xminus_hits[:, 2],
        # marker_color='blue',
        name='edge cathode xminus hits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=-edge_cathode_xminus_hits[:, 0],
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>dist:%{customdata:.3f}',
        )
fig.add_traces(PHits_cathode_xminus_traces)

PHits_ct_ends_traces = go.Scatter3d(
        x=ct_ends[:, 0], y=ct_ends[:, 1], z=ct_ends[:, 2],
        # marker_color='blue',
        name='cluster ends',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>',
        )
fig.add_traces(PHits_ct_ends_traces)


# fig.add_traces(ct_hits)
# fig.add_traces(PCA_mean)
# fig.add_traces(PCA_line)
# fig.add_traces(seg_hits)
# fig.add_traces(seg_PCA_line)
# fig.add_traces(seg_PCA_mean)

fig.add_traces(PHits_traces)

fig.update_layout(
    width=1024, height=768,
    legend_orientation="h",
    scene = dict(xaxis_title='x [cm]',
                yaxis_title='y [cm]',
                zaxis_title='z [cm]')
)


fig.show()


In [ ]:
dir_ana = "/global/cfs/cdirs/dune/www/data/ModuleX/analysis_data/analysis_0-5_Efield_1/"

In [ ]:
all_edge_xmax_hits = []
all_edge_xmin_hits = []
all_edge_ymax_hits = []
all_edge_ymin_hits = []
all_edge_zmax_hits = []
all_edge_zmin_hits = []
all_edge_cathode_xplus_hits = []
all_edge_cathode_xminus_hits = []

all_ct_ends = []

all_ct_fit_length = []
all_ct_PCA_mean = []
all_ct_PCA_vec = []
all_ct_PCA_variance = []
all_ct_PCA_singular_values = []
all_ct_pts_density = []

all_seg_fit_length = []
all_seg_PCA_mean = []
all_seg_PCA_vec = []
all_seg_PCA_variance = []
all_seg_PCA_singular_values = []
all_seg_pts_density = []
all_seg_cos_angle = []
all_seg_angle = []
all_seg_joint = []

for file_name in os.listdir(dir_ana):
    # print(file_name)
    f = np.load(f"{dir_ana}{file_name}")
    all_edge_xmax_hits.append(f["edge_xmax_hits"])
    all_edge_xmin_hits.append(f["edge_xmin_hits"])
    all_edge_ymax_hits.append(f["edge_ymax_hits"])
    all_edge_ymin_hits.append(f["edge_ymin_hits"])
    all_edge_zmax_hits.append(f["edge_zmax_hits"])
    all_edge_zmin_hits.append(f["edge_zmin_hits"])
    all_edge_cathode_xplus_hits.append(f["edge_cathode_xplus_hits"])
    all_edge_cathode_xminus_hits.append(f["edge_cathode_xminus_hits"])
    
    all_ct_ends.append(f['ct_ends'])
    
    all_ct_fit_length.append(f["ct_fit_length"])
    all_ct_PCA_mean.append(f["ct_PCA_mean"])
    all_ct_PCA_vec.append(f["ct_PCA_vec"])
    all_ct_PCA_variance.append(f["ct_PCA_variance"])
    all_ct_PCA_singular_values.append(f["ct_PCA_singular_values"])
    all_ct_pts_density.append(f["ct_pts_density"])
    
    all_seg_fit_length.append(f["seg_fit_length"])
    all_seg_PCA_mean.append(f["seg_PCA_mean"])
    all_seg_PCA_vec.append(f["seg_PCA_vec"])
    all_seg_PCA_variance.append(f["seg_PCA_variance"])
    all_seg_PCA_singular_values.append(f["seg_PCA_singular_values"])
    all_seg_pts_density.append(f["seg_pts_density"])
    all_seg_cos_angle.append(f["seg_cos_angle"])
    all_seg_angle.append(f["seg_angle"])
    all_seg_joint.append(f["seg_joint"])
    
all_edge_xmax_hits = np.concatenate(all_edge_xmax_hits)
all_edge_xmin_hits = np.concatenate(all_edge_xmin_hits)
all_edge_ymax_hits = np.concatenate(all_edge_ymax_hits)
all_edge_ymin_hits = np.concatenate(all_edge_ymin_hits)
all_edge_zmax_hits = np.concatenate(all_edge_zmax_hits)
all_edge_zmin_hits = np.concatenate(all_edge_zmin_hits)
all_edge_cathode_xplus_hits = np.concatenate(all_edge_cathode_xplus_hits)
all_edge_cathode_xminus_hits = np.concatenate(all_edge_cathode_xminus_hits)

all_ct_ends = np.concatenate(all_ct_ends)

all_ct_fit_length = np.concatenate(all_ct_fit_length)
all_ct_PCA_mean = np.concatenate(all_ct_PCA_mean)
all_ct_PCA_vec = np.concatenate(all_ct_PCA_vec)
all_ct_PCA_variance = np.concatenate(all_ct_PCA_variance)
all_ct_PCA_singular_values = np.concatenate(all_ct_PCA_singular_values)
all_ct_pts_density = np.concatenate(all_ct_pts_density)

all_seg_fit_length = np.concatenate(all_seg_fit_length)
all_seg_PCA_mean = np.concatenate(all_seg_PCA_mean)
all_seg_PCA_vec = np.concatenate(all_seg_PCA_vec)
all_seg_PCA_variance = np.concatenate(all_seg_PCA_variance)
all_seg_PCA_singular_values = np.concatenate(all_seg_PCA_singular_values)
all_seg_pts_density = np.concatenate(all_seg_pts_density)
all_seg_cos_angle = np.concatenate(all_seg_cos_angle)
all_seg_angle = np.concatenate(all_seg_angle)
all_seg_joint = np.concatenate(all_seg_joint)

In [ ]:
plt.hist(all_seg_pts_density, bins=100)
plt.yscale("log")
plt.show()

In [ ]:
plt.hist(all_seg_angle, bins=100)
plt.xlabel("angular differences between neighbouring segments")
plt.ylabel("number of segments")
plt.savefig("plots/segment_anglular_differences.pdf")
plt.show()

In [ ]:
plt.hist(all_ct_PCA_variance[:, 1], bins=200)
plt.yscale("log")
plt.xlabel("variance of the 2nd principal axis")
plt.ylabel("number of tracks")
plt.savefig("plots/track_2nd_PCA_variance.pdf")
plt.show()

In [ ]:
plt.hist(all_ct_pts_density, bins=200)
plt.yscale('log')
plt.show()

In [ ]:
fig = go.Figure()

all_ct_ends = all_ct_ends[(all_ct_ends[:, 0] > (min(x_boundaries)+edge_threshold)) & (all_ct_ends[:, 0] < (max(x_boundaries)-edge_threshold))]
##########################
# Draw the cathodes
##########################
fig.add_traces(draw_cathode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.3,
    colorscale='Greys',
))

##########################
# Draw the anodes
##########################
fig.add_traces(draw_anode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

PHits_ct_ends_traces = go.Scatter3d(
        x=all_ct_ends[:, 0], y=all_ct_ends[:, 1], z=all_ct_ends[:, 2],
        # marker_color='blue',
        name='cluster ends',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>',
        )
fig.add_traces(PHits_ct_ends_traces)

fig.update_layout(
    width=1024, height=768,
    legend_orientation="h",
    scene = dict(xaxis_title='x [cm]',
                yaxis_title='y [cm]',
                zaxis_title='z [cm]')
)


fig.show()

In [ ]:
all_seg_joint_ft[:200]

In [ ]:
len(all_seg_joint_ft[:, 0])

In [ ]:
len(all_seg_joint[:, 0])

In [ ]:
all_seg_joint_ft[:, 1]

In [ ]:
all_seg_joint_ft[:, 2]

In [ ]:
############
# x max
############
h_xmax_total, zedges, yedges = np.histogram2d(all_edge_xmax_hits[:,2], all_edge_xmax_hits[:,1], weights=max(x_boundaries)-all_edge_xmax_hits[:,0], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_xmax_count, zedges, yedges = np.histogram2d(all_edge_xmax_hits[:,2], all_edge_xmax_hits[:,1], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_xmax = h_xmax_total / h_xmax_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,10))
c_xmax = plt.pcolormesh(Z, Y, h_xmax.T)
plt.colorbar(c_xmax)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("projected gap to the edge of max x (TPC 2)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/x_max_dist.pdf")
plt.show()

plt.figure(figsize=(6,10))
c_xmax_count = plt.pcolormesh(Z, Y, h_xmax_count.T, vmax=300)
plt.colorbar(c_xmax_count)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/x_max_stat.pdf")
plt.show()

############
# x min
############
h_xmin_total, zedges, yedges = np.histogram2d(all_edge_xmin_hits[:,2], all_edge_xmin_hits[:,1], weights=all_edge_xmin_hits[:,0]-min(x_boundaries), bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_xmin_count, zedges, yedges = np.histogram2d(all_edge_xmin_hits[:,2], all_edge_xmin_hits[:,1], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_xmin = h_xmin_total / h_xmin_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,10))
c_xmin = plt.pcolormesh(Z, Y, h_xmin.T)
plt.colorbar(c_xmin)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("projected gap to the edge of min x (TPC 1)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/x_min_dist.pdf")
plt.show()

plt.figure(figsize=(6,10))
c_xmin_count = plt.pcolormesh(Z, Y, h_xmin_count.T, vmax=300)
plt.colorbar(c_xmin_count)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/x_min_stat.pdf")
plt.show()

############
# cathode xplus
############
all_edge_cathode_xplus_hits_filter = all_edge_cathode_xplus_hits[all_edge_cathode_xplus_hits[:,1] > max(y_boundaries)/2]
h_cathode_xplus_total, zedges, yedges = np.histogram2d(all_edge_cathode_xplus_hits_filter[:,2], all_edge_cathode_xplus_hits_filter[:,1], weights=all_edge_cathode_xplus_hits_filter[:, 0], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_cathode_xplus_count, zedges, yedges = np.histogram2d(all_edge_cathode_xplus_hits_filter[:,2], all_edge_cathode_xplus_hits_filter[:,1], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_cathode_xplus = h_cathode_xplus_total / h_cathode_xplus_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,10))
c_cathode_xplus = plt.pcolormesh(Z, Y, h_cathode_xplus.T)
plt.colorbar(c_cathode_xplus)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("projected gap to the cathode x plus (TPC 2)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/cathode_xplus_dist.pdf")
plt.show()

plt.figure(figsize=(6,10))
c_cathode_xplus_count = plt.pcolormesh(Z, Y, h_cathode_xplus_count.T, vmax=300)
plt.colorbar(c_cathode_xplus_count)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/cathode_xplus_stat.pdf")
plt.show()

############
# cathode xminus
############
h_cathode_xminus_total, zedges, yedges = np.histogram2d(all_edge_cathode_xminus_hits[:,2], all_edge_cathode_xminus_hits[:,1], weights=-all_edge_cathode_xminus_hits[:, 0], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_cathode_xminus_count, zedges, yedges = np.histogram2d(all_edge_cathode_xminus_hits[:,2], all_edge_cathode_xminus_hits[:,1], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_cathode_xminus = h_cathode_xminus_total / h_cathode_xminus_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,10))
c_cathode_xminus = plt.pcolormesh(Z, Y, h_cathode_xminus.T)
plt.colorbar(c_cathode_xminus)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("projected gap to the cathode x minus (TPC 1)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/cathode_xminus_dist.pdf")
plt.show()

plt.figure(figsize=(6,10))
c_cathode_xminus_count = plt.pcolormesh(Z, Y, h_cathode_xminus_count.T, vmax=300)
plt.colorbar(c_cathode_xminus_count)
plt.xlabel("z [cm]")
plt.ylabel("y [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/cathode_xminus_stat.pdf")
plt.show()

############
# y max
############
h_ymax_total, zedges, yedges = np.histogram2d(all_edge_ymax_hits[:,0], all_edge_ymax_hits[:,2], weights=max(y_boundaries)-all_edge_ymax_hits[:,1], bins=[140,140], range=[[x_boundaries[0],x_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
h_ymax_count, zedges, yedges = np.histogram2d(all_edge_ymax_hits[:,0], all_edge_ymax_hits[:,2], bins=[140,140], range=[[x_boundaries[0],x_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
h_ymax = h_ymax_total / h_ymax_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,5))
c_ymax = plt.pcolormesh(Z, Y, h_ymax.T)
plt.colorbar(c_ymax)
plt.xlabel("x [cm]")
plt.ylabel("z [cm]")
plt.title("projected gap to the edge of max y (top)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/ymax_dist.pdf")
plt.show()

plt.figure(figsize=(6,5))
c_ymax_count = plt.pcolormesh(Z, Y, h_ymax_count.T, vmax=300)
plt.colorbar(c_ymax_count)
plt.xlabel("x [cm]")
plt.ylabel("z [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/ymax_stat.pdf")
plt.show()

############
# y min
############
h_ymin_total, zedges, yedges = np.histogram2d(all_edge_ymin_hits[:,0], all_edge_ymin_hits[:,2], weights=all_edge_ymin_hits[:,1]-min(y_boundaries), bins=[140,140], range=[[x_boundaries[0],x_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
h_ymin_count, zedges, yedges = np.histogram2d(all_edge_ymin_hits[:,0], all_edge_ymin_hits[:,2], bins=[140,140], range=[[x_boundaries[0],x_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
h_ymin = h_ymin_total / h_ymin_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,5))
c_ymin = plt.pcolormesh(Z, Y, h_ymin.T)
plt.colorbar(c_ymin)
plt.xlabel("x [cm]")
plt.ylabel("z [cm]")
plt.title("projected gap to the edge of bottom (bottom)")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/ymin_dist.pdf")
plt.show()

plt.figure(figsize=(6,5))
c_ymin_count = plt.pcolormesh(Z, Y, h_ymin_count.T, vmax=300)
plt.colorbar(c_ymin_count)
plt.xlabel("x [cm]")
plt.ylabel("z [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/ymin_stat.pdf")
plt.show()

############
# z min
############
h_zmin_total, zedges, yedges = np.histogram2d(all_edge_zmin_hits[:,0], all_edge_zmin_hits[:,1], weights=all_edge_zmin_hits[:,2]-min(z_boundaries), bins=[140,280], range=[[x_boundaries[0],x_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_zmin_count, zedges, yedges = np.histogram2d(all_edge_zmin_hits[:,0], all_edge_zmin_hits[:,1], bins=[140,280], range=[[x_boundaries[0],x_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
h_zmin = h_zmin_total / h_zmin_count
Z, Y = np.meshgrid(zedges, yedges)
plt.figure(figsize=(6,10))
c_zmin = plt.pcolormesh(Z, Y, h_zmin.T)
plt.colorbar(c_zmin)
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
plt.title("projected gap to the edge of min z")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/zmin_dist.pdf")
plt.show()

plt.figure(figsize=(6,10))
c_zmin_count = plt.pcolormesh(Z, Y, h_zmin_count.T, vmax=300)
plt.colorbar(c_zmin_count)
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
plt.title("number of hits per bin")
plt.margins(0.2)
plt.set_cmap("Greens")
plt.savefig(f"plots/TPC_edge/zmin_stat.pdf")
plt.show()

In [ ]:
fig = go.Figure()

##########################
# Draw the cathodes
##########################
fig.add_traces(draw_cathode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.3,
    colorscale='Greys',
))

##########################
# Draw the anodes
##########################
fig.add_traces(draw_anode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

PHits_ct_ends_traces = go.Scatter3d(
        x=ct_ends[:, 0], y=ct_ends[:, 1], z=ct_ends[:, 2],
        # marker_color='blue',
        name='cluster ends',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>',
        )
fig.add_traces(PHits_ct_ends_traces)

fig.update_layout(
    width=1024, height=768,
    legend_orientation="h",
    scene = dict(xaxis_title='x [cm]',
                yaxis_title='y [cm]',
                zaxis_title='z [cm]')
)


fig.show()